In [ ]:
import pandas, numpy, seaborn as pd, np, sb
import matplotlib.pyplot as plt

In [1]:
!pip install openai pandas tqdm



In [ ]:
# Step 2: Import Libraries
import openai
import pandas as pd
from tqdm import tqdm
import time

# Step 3: Set OpenAI API Key
from openai import OpenAI
apikey = <apikey>
openai.api_key = apikey  # Replace with your actual key



In [4]:
df = pd.read_csv("SMG_text_rating_global.csv")  # Replace with your file path
df = df.drop_duplicates().dropna(subset=['ANSWER_VALUE_EN'])
df

,Filter_Brand,Filter_BMU,Filter_Country,INTERNAL_STORE_NUMBER,REVIEW_DATE,CHAMPS_NUMBER,ANSWER_VALUE_EN
0,PH,GFMSA,IDN - Indonesia,1372637,2025-06-01,62PR346,"""(Translated by google) it's so much fun at pi..."
2,PH,GFMSA,IDN - Indonesia,1437416,2025-06-01,62PC084,"""Good services"""
3,PH,GFMSA,IDN - Indonesia,13201,2025-06-01,62PR081,"""Great vibes great service 👌🙌"""
5,PH,GFMSA,IDN - Indonesia,38948,2025-06-01,62PR267,"""(Translated by Google) all service is good & ..."
7,PH,GFMSA,IDN - Indonesia,38948,2025-06-01,62PR267,"""(Translated by google) I have eaten here many..."
...,...,...,...,...,...,...,...
186082,PH,India,IND - India,40706,2025-06-13,91PDIPL220-01,"""Very good service and quality"""
186084,PH,GFMMEP,ARE - United Arab Emirates,13443,2025-06-13,971P16738,"""Love pizza"""
186085,PH,GFMSA,IDN - Indonesia,53745,2025-06-13,62PP120,"""The pizza tebel is getting more stable"""
186119,PH,GFMLA&C,BRA - Brazil,1385066,2025-06-01,55P75533,"""I really liked the pizza, for? M, Margueritta..."


In [19]:
df1 = df.groupby('CHAMPS_NUMBER')['ANSWER_VALUE_EN'].apply(''.join).reset_index().head(10)
df1.columns = ['CHAMPS_NUMBER', 'Text']

len(df1['Text'][0:1][0])






165

In [35]:

# Step 5: Define Prompt Template
prompt_template = """
You are an assistant for summarizing customer comments about a restaurant. 
Each comment should be summarized into the following fields:
- Food quality:< popularity in percentage> + <summary>
- Service quality:< popularity in percentage> + <summary>
- Cleanliness:< popularity in percentage> + <summary>
- Overall sentiment (positive/negative/neutral):

Comment: "{Text}"
Summary:
"""

# Step 6: Summarization Function using GPT-4o
def summarize_comment(Text, model="gpt-4o"):
    prompt = prompt_template.format(Text=Text)
    try:
        client = openai.OpenAI(api_key=apikey)
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=200
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error for comment: {Text[:30]}... - {e}")
        return ""


In [36]:

# Step 7: Apply Summarization in Batch
tqdm.pandas()
df1['summary'] = df1['Text'].progress_apply(lambda x: summarize_comment(x))
df1

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:29<00:00,  2.99s/it]


,CHAMPS_NUMBER,Text,summary
0,1340P22,"""Line to pickup is always long but that’s norm...",- Food quality: 100% + The pan pepperoni pizza...
1,1809P01,"""Excellent experience, good service""""Excellent...",- Food quality: 100% + The pizza is highly pra...
2,1809P02,"""I enjoyed it very much""""Super rich, fresh ing...",- Food quality: 100% + The comment highlights ...
3,1809P03,"""I really liked service and very friendly staf...","Sure, here is the summary of the customer comm..."
4,1809P07,"""Normal very cordial""""Excellent customer servi...",- Food quality: 90% + Most customers found the...
5,1809P08,"""All good""""Really satisfied very good attentio...",- Food quality: 100% + The comment indicates s...
6,1809P09,"""Excellent""""I felt satisfied with the staff by...",- Food quality: 80% + Many customers praised t...
7,1809P10,"""Very good""""The collaborator was very kind""",- Food quality: Not mentioned\n- Service quali...
8,1809P11Relo,"""Very good""""Excellent!""""Very good service""""I l...",- Food quality: 100% + The comments do not spe...
9,1809P12,"""Excellent service""""Excellent service""""Very go...","- Food quality: 80% + The food, particularly t..."


In [37]:
# Display full text in notebook output
pd.set_option('display.max_colwidth', None)  # or use a large number like 1000
df1

,CHAMPS_NUMBER,Text,summary
0,1340P22,"""Line to pickup is always long but that’s normal for island life. The pan pepperoni pizza was excellent. Lasagna and spaghetti and meatballs are always really good!""","- Food quality: 100% + The pan pepperoni pizza was excellent, and the lasagna and spaghetti with meatballs are consistently really good.\n- Service quality: 70% + The line to pick up orders is always long, which is typical for island life.\n- Cleanliness: Not mentioned\n- Overall sentiment: Positive"
1,1809P01,"""Excellent experience, good service""""Excellent service by staff, they are very friendly and professionals""""They treated me super well""""Good service in the dining room, fast and efficient, the waitress Jimura attended to me with a Japanese name""""In general it was very good""""The best pizza in the country 100%""","- Food quality: 100% + The pizza is highly praised, described as the best in the country.\n- Service quality: 100% + The service is consistently described as excellent, with friendly and professional staff. Specific mention of fast and efficient service in the dining room, with a positive note on the waitress Jimura.\n- Cleanliness: Not explicitly mentioned in the comments, so no percentage or summary can be provided.\n- Overall sentiment: Positive"
2,1809P02,"""I enjoyed it very much""""Super rich, fresh ingredients (translated by google) super delicious, fresh ingredients""""GOOD""","- Food quality: 100% + The comment highlights that the food was ""super delicious"" and made with ""fresh ingredients,"" indicating high satisfaction with the food quality.\n- Service quality: Not mentioned\n- Cleanliness: Not mentioned\n- Overall sentiment: Positive"
3,1809P03,"""I really liked service and very friendly staff I am satisfied""""Excellent service""""Very good""""It was great !!!""""Very friendly and quite oriented staff with my order. Very good service""""It was greatly satisfactory and delicious""""Very pleasant""""Excellent service""""It was fine, if I would like more personal combos options.""""Very customer service""""Excellent service""""Delighted with everything in general""","Sure, here is the summary of the customer comments:\n\n- **Food quality: 70%** + The comments generally reflect satisfaction with the food, describing it as ""delicious"" and ""satisfactory,"" though one comment suggests a desire for more personalized combo options.\n\n- **Service quality: 90%** + The service is highly praised, with frequent mentions of ""excellent service,"" ""very friendly staff,"" and ""customer-oriented"" interactions. The staff is described as friendly and attentive, contributing to a positive dining experience.\n\n- **Cleanliness: 0%** + There are no specific mentions of cleanliness in the comments provided.\n\n- **Overall sentiment: Positive** + The overall sentiment is overwhelmingly positive, with multiple expressions of satisfaction and delight with both the service and the food."
4,1809P07,"""Normal very cordial""""Excellent customer service, very personalized care with excellent recommendations""""GISMELL Cashier of Carry Out very good""""Very satisfied""""Excellent""""Excellent staff in Zona Rosa, Santiago""""Very good service and was very good""""Excellent service The girls who attended to us Laura and Gismel are willing to help make your order successfully""""Very good pizza, excellent service,""""Lllll""""All good as always, except noise.""","- Food quality: 90% + Most customers found the food, particularly the pizza, to be very good.\n- Service quality: 95% + The service was frequently praised as excellent, with personalized care and helpful staff, particularly mentioning Laura and Gismel.\n- Cleanliness: 80% + Cleanliness was generally perceived as good, though not specifically highlighted in many comments.\n- Overall sentiment: Positive"
5,1809P08,"""All good""""Really satisfied very good attention for me and my family""","- Food quality: 100% + The comment indicates satisfaction, sugge